# Amazon Review EDA

General EDA to understand the data, and create potential avenues for a model. Data from [this Amazon Product Dataset on Kaggle](https://www.kaggle.com/datasets/piyushjain16/amazon-product-data) - note that no meta card was provided.

In [1]:
# Modules
import pandas as pd
import numpy as np
from pathlib import Path
import os

In [3]:
# Load in data
try:
    reviews = pd.read_csv(Path(os.getcwd()).parents[0].joinpath("data", "amazon_reviews.csv"))
except FileNotFoundError:
    reviews = pd.read_csv(Path(os.getcwd()).parents[0].joinpath("data", "train.csv"))

Took 18 seconds to load on OP computer.
On laptop, it took 1m21s! Geez...

## Data Exploration

Because there isn't a metacard provided, let's do some old-fashioned EDA

In [7]:
reviews.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000


In [6]:
# View head
print(f"Data dimensions: {reviews.shape}")

Data dimensions: (2249698, 6)


We have 6 columns with 2,249,698 rows (products). Columns include a `PRODUCT_ID`, `TITLE`, `BULLET_POINTS`, `DESCRIPTION`, `PRODUCT_TYPE_ID`, and `PRODUCT_LENGTH`.